In [ ]:
!apt install ffmpeg
!pip install colabcode
!pip install fastapi

In [ ]:
pip install spleeter

In [ ]:
pip install pyrebase

In [ ]:
pip install python-multipart

In [ ]:
from IPython.display import Audio
from colabcode import ColabCode
cc = ColabCode(port=12000, code=False)

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok   

In [ ]:
!ngrok authtoken #enter ngrok authtoken here

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import files
import os
import subprocess
from fastapi import FastAPI, File, UploadFile, Request, Form
import shutil
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pyrebase



app = FastAPI()

config = {
    #add firebase config details here
    }

firebase = pyrebase.initialize_app(config)    
storage = firebase.storage()    

origins = [
    "http://localhost:3000/",
    "http://localhost:3000"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=False,
    allow_methods=["*"],
    allow_headers=["*"],
)



@app.get("/")
async def read_root():
  print("Came")
  return "HEllo World"

@app.post("/data")
async def test(item : Request):
  data = await item.json()
  return {"d" : data}



@app.post("/upload")
def upload_file(audio_file: UploadFile = File(...), uid: str = Form(...)):
  filename = audio_file.filename
  print(filename)

  with open(filename, "wb") as buffer:
    shutil.copyfileobj(audio_file.file, buffer)

  try:
    stream = subprocess.run('spleeter separate -o output/ ' + filename, shell = True)
    print(type(stream))
    vocals = "output/" + filename.split(".")[0] + "/vocals.wav"
    #print(type(vocals))
    storage.child(uid + "/vocals/" + filename).put(vocals)

  except Exception as e:
    print(f"{e}")

  return "Success"

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://8ddc-34-86-245-109.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [59]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Rockabye.mp3
<class 'subprocess.CompletedProcess'>
[Errno 2] No such file or directory: 'output/Rockabye/vocals.wav'
INFO:     103.134.130.206:0 - "POST /upload HTTP/1.1" 200 OK


In [ ]:
Audio("output/Sunflower/vocals.wav")